In [0]:
import tensorflow as tf
import cv2
import glob
import random
import numpy as np
import pandas as pd
import csv
import time
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Import data from Kaggle site

In [0]:
from google.colab import files
files.upload()

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls -l ~/.kaggle
!cat ~/.kaggle/kaggle.json

In [0]:
!pip install -q kaggle
!pip install -q kaggle-cli
!kaggle competitions download -c severstal-steel-defect-detection

In [0]:
!unzip train.csv.zip
!unzip train_images.zip -d train_images
!unzip test_images.zip -d test_images

# Preprocess masks

### marker-to-mask converter

In [0]:
def make_mask(marker_string, nv, nh):
    # initial mask is empty 1D array
    mask = np.zeros(nv * nh)
    
    # if markers exist, add detect part according to markers
    if len(marker_string) > 0:
        markers = np.array(marker_string.split(' ')).reshape(-1, 2)
        for marker in markers:
            start = int(marker[0])
            length = int(marker[1])
            mask[start: start+length] = 1.0
            
    mask = np.reshape(mask, (nv, nh))

    return mask            

### extract the first N image IDs and their masks

In [0]:
N = 2000 # read the top N rows from file => will get N/4 cases
D0 = 256
D1 = 1600
BATCH_SIZE = 20

train_mask_file = "train.csv"

with open(train_mask_file, "r", newline="") as f:
    reader = csv.reader(f)
    header = next(reader)
    print("CSV header: {}".format(header))
    
    masks = dict()
    for i, row in enumerate(reader):
        if i < N:
            # read image id and defect type
            img_id, defect_type = row[0].split('.jpg_')
        
            # process mask
            mask_marker_string = row[1]
            mask = make_mask(mask_marker_string, D0, D1)
        
            if img_id in masks:
                # if "masks" already has this image's mask(s), add this mask to the right channel
                masks[img_id][:, :, int(defect_type) - 1] = mask
            else:
                # if "masks" doesn't contain this image's info, create a 4-channel mask with zeros
                masks[img_id] = np.zeros((D0, D1, 4))  # dict({defect_type: mask})

# Preprocess training images

## Import images

In [0]:
train_img_dir = 'train_images/'
train_img_suffix = '.jpg'

def get_image(img_id):
    image_path = train_img_dir + img_id + train_img_suffix
    # import image
    image = mpimg.imread(image_path)
    image = image[:, :, 0:1] # grayscale -> pick just one channel
    image = image / 255.0
    return image

## Create dataset

In [0]:
# Only deal with one type of defect at a time
defect_type = 0

image_list = []
mask_list = []
serial_list = []
for serial, mask in masks.items():
    image = get_image(serial)
    image_list = image_list + [image]
    mask_list = mask_list + [mask[:, :, defect_type: defect_type + 1]]
    serial_list = serial_list + [serial]

ds = tf.data.Dataset.from_tensor_slices((np.array(image_list), np.array(mask_list)))
ds = ds.batch(batch_size=BATCH_SIZE)
ds = ds.shuffle(buffer_size=200)

# Set up model

## Define model

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=16, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(strides=2),
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(strides=2),
    tf.keras.layers.Conv2DTranspose(1, kernel_size=3, strides=4, padding='same', activation='relu')
])

## Configure training

In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
model_history = model.fit(ds, epochs=5)

In [0]:
pred = model.predict(ds.take(5))